# Calculate PMV values (V1)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime
import time

current_dir

'C:\\Users\\juliu\\Desktop\\pmv-air-conditioning-control\\code\\src'

In [3]:
from pmv_formula import calculate_PMV_value

In [4]:
PMV, PPD = calculate_PMV_value(Temp = 24, MeanR = 24, VEL = 0.45 , hum = 45, MET = 1.0, CLO = 1.0, EXW = 0.0, WV = 0.0)

In [5]:
print(PMV)

-0.3260912060822618


In [6]:
print(PPD)

7.211077770619937


In [7]:
# create dataframe for new PMV values

data = pd.DataFrame()
data['entry_id'] = pd.Series()
data['Temperature'] = pd.Series()
data['Humidity'] = pd.Series()
data['Air_velocity'] = pd.Series()
#data['mrt'] = pd.Series()
data['met'] = pd.Series()
data['clo'] = pd.Series()
data['PMV'] = pd.Series()
data['PPD'] = pd.Series()

# data.set_index('entry_id', inplace=True)

# data

,entry_id,Temperature,Humidity,Air_velocity,met,clo,PMV,PPD


In [8]:
# GENERATION OF TABLE WITH PMV DATA (calculated by Fanger Equation for Thermal Comfort)

start_time = time.time()

# start, end, step
entry_id = 1

# values for function call that we don't use atm
external_work = 0
water_vapour_pressure = 0

# temperature
temp_min = 16
temp_max = 30
temp_step = 0.1

# humidity
hum_min = 30
hum_max = 80
hum_step = 1

# air velocity
air_vel_min = 0
air_vel_max = 0.7
air_vel_step = 0.05

# met 
met_min = 1 #0.5
met_max = 1 #2
met_step = 1 #0.5

# clo
clo_min = 1 #0.5
clo_max = 1 #2
clo_step = 1 #0.5



for temp_val in np.arange(temp_min, temp_max + temp_step, temp_step):
    for hum_val in np.arange(hum_min, hum_max + hum_step, hum_step):
        for air_vel_val in np.arange(air_vel_min, air_vel_max + air_vel_step, air_vel_step):
            for met_val in np.arange(met_min, met_max + met_step, met_step):
                for clo_val in np.arange(clo_min, clo_max + clo_step, clo_step):
                    
                    PMV_val, PPD_val = calculate_PMV_value(Temp = temp_val, MeanR = temp_val, VEL = air_vel_val , hum = hum_val, MET = met_val, CLO = clo_val, EXW = external_work, WV = water_vapour_pressure)
                    
                    data.at[entry_id - 1, 'entry_id'] = entry_id
                    data.at[entry_id - 1, 'Temperature'] = temp_val
                    data.at[entry_id - 1, 'Humidity'] = hum_val
                    data.at[entry_id - 1, 'Air_velocity'] = air_vel_val
                    data.at[entry_id - 1, 'met'] = met_val
                    data.at[entry_id - 1, 'clo'] = clo_val
                    data.at[entry_id - 1, 'PMV'] = PMV_val
                    data.at[entry_id - 1, 'PPD'] = PPD_val
                    
                    
                    if not(entry_id % 10000):
                        print(entry_id)
                    entry_id += 1
                   

    
data.set_index('entry_id', inplace=True)

print (entry_id)

stop_time = time.time()

time = stop_time-start_time

print('Calculation time: ' + str(round(time, 2)) + ' sec')

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
107866
Calculation time: 498.55 sec


In [11]:
data.round(2)

,Temperature,Humidity,Air_velocity,met,clo,PMV,PPD
entry_id,,,,,,,
1.0,16.0,30.0,0.00,1.0,1.0,-2.12,81.83
2.0,16.0,30.0,0.05,1.0,1.0,-2.12,81.83
3.0,16.0,30.0,0.10,1.0,1.0,-2.12,81.83
4.0,16.0,30.0,0.15,1.0,1.0,-2.28,87.72
5.0,16.0,30.0,0.20,1.0,1.0,-2.43,91.76
...,...,...,...,...,...,...,...
107861.0,30.0,80.0,0.50,1.0,1.0,1.97,75.27
107862.0,30.0,80.0,0.55,1.0,1.0,1.96,74.73
107863.0,30.0,80.0,0.60,1.0,1.0,1.95,74.24


In [14]:
data.round(2).to_csv('output_PMV_PPD_data.csv')

# data.round(2).to_csv(str(datetime.date.today()) + '_output_PMV_PPD_data.csv')